In [9]:
import gradio as gr
import joblib

In [13]:

heart_model = joblib.load('heart.joblib')
diabetes_model = joblib.load('diabetes.joblib')
kidney_model = joblib.load('kidney.joblib')
heart_rate_model = joblib.load('heart_rate.joblib')

In [16]:
import gradio as gr
import numpy as np
import joblib


# Define Prediction Functions with Exception Handling and Type Conversion
def predict_heart(age=None, resting_bp=None, cholesterol=None, sex=None, chest_pain_type=None, resting_ecg=None, exercise_angina=None, st_slope=None, fasting_bs=None):
    try:
        input_data = [[age, resting_bp, cholesterol, sex, chest_pain_type, resting_ecg, exercise_angina, st_slope, fasting_bs]]
        input_data = np.nan_to_num(input_data, nan=-1)
        prediction = heart_model.predict(input_data)
        return int(prediction[0])
    except Exception as e:
        return {'error': str(e)}

def predict_diabetes(pregnancies=None, glucose=None, blood_pressure=None, skin_thickness=None, insulin=None, bmi=None, diabetes_pedigree_function=None, age=None):
    try:
        input_data = [[pregnancies, glucose, blood_pressure, skin_thickness, insulin, bmi, diabetes_pedigree_function, age]]
        input_data = np.nan_to_num(input_data, nan=-1)
        prediction = diabetes_model.predict(input_data)
        return int(prediction[0])
    except Exception as e:
        return {'error': str(e)}

def predict_kidney(age=None, bp=None, su=None, al=None, bgr=None, bu=None, sc=None, hemo=None, pcc=None, ba=None, htn=None, dm=None, cad=None, appet=None, ane=None):
    try:
        input_data = [[age, bp, su, al, bgr, bu, sc, hemo, pcc, ba, htn, dm, cad, appet, ane]]
        input_data = np.nan_to_num(input_data, nan=-1)
        prediction = kidney_model.predict(input_data)
        return int(prediction[0])
    except Exception as e:
        return {'error': str(e)}

def predict_framingham(age=None, totChol=None, BMI=None, heartRate=None, glucose=None, sysBP=None, diaBP=None, cigsPerDay=None, gender=None, education=None, current_smoker=None, age_groups=None):
    try:
        input_data = [[age, totChol, BMI, heartRate, glucose, sysBP, diaBP, cigsPerDay, gender, education, current_smoker, age_groups]]
        input_data = np.nan_to_num(input_data, nan=-1)
        prediction = framingham_model.predict(input_data)
        return int(prediction[0])
    except Exception as e:
        return {'error': str(e)}

# Model selection
model_choice = gr.inputs.CheckboxGroup(choices=['Heart Disease', 'Diabetes', 'Kidney Disease', 'Framingham Risk'], label="Choose Models to Predict")

# Define input elements
heart_inputs = [
    gr.inputs.Number(label="Age"),
    gr.inputs.Number(label="Resting Blood Pressure"),
    gr.inputs.Number(label="Cholesterol"),
    gr.inputs.Dropdown(choices=['Male', 'Female'], label="Sex"),
    gr.inputs.Dropdown(choices=['Type 1', 'Type 2', 'Type 3', 'Type 4'], label="Chest Pain Type"),
    gr.inputs.Dropdown(choices=['Normal', 'Abnormal'], label="Resting ECG"),
    gr.inputs.Checkbox(label="Exercise Angina"),
    gr.inputs.Dropdown(choices=['Upsloping', 'Flat', 'Downsloping'], label="ST Slope"),
    gr.inputs.Checkbox(label="Fasting BS")
]

diabetes_inputs = [
    gr.inputs.Number(label="Pregnancies"),
    gr.inputs.Number(label="Glucose"),
    gr.inputs.Number(label="Blood Pressure"),
    gr.inputs.Number(label="Skin Thickness"),
    gr.inputs.Number(label="Insulin"),
    gr.inputs.Number(label="BMI"),
    gr.inputs.Number(label="Diabetes Pedigree Function"),
    gr.inputs.Number(label="Age")
]

kidney_inputs = [
    gr.inputs.Number(label="Age"),
    gr.inputs.Number(label="Blood Pressure"),
    gr.inputs.Number(label="Sugar Level"),
    gr.inputs.Number(label="Albumin Level"),
    gr.inputs.Number(label="Blood Glucose Random"),
    gr.inputs.Number(label="Blood Urea"),
    gr.inputs.Number(label="Serum Creatinine"),
    gr.inputs.Number(label="Hemoglobin"),
    gr.inputs.Dropdown(choices=['Present', 'Not Present'], label="Pus Cell Clumps"),
    gr.inputs.Dropdown(choices=['Bacteria', 'No Bacteria'], label="Bacteria"),
    gr.inputs.Checkbox(label="Hypertension"),
    gr.inputs.Checkbox(label="Diabetes Mellitus"),
    gr.inputs.Checkbox(label="Coronary Artery Disease"),
    gr.inputs.Dropdown(choices=['Good', 'Poor'], label="Appetite"),
    gr.inputs.Checkbox(label="Anemia")
]

framingham_inputs = [
    gr.inputs.Number(label="Age"),
    gr.inputs.Number(label="Total Cholesterol"),
    gr.inputs.Number(label="BMI"),
    gr.inputs.Number(label="Heart Rate"),
    gr.inputs.Number(label="Glucose"),
    gr.inputs.Number(label="Systolic BP"),
    gr.inputs.Number(label="Diastolic BP"),
    gr.inputs.Number(label="Cigarettes per Day"),
    gr.inputs.Dropdown(choices=['Male', 'Female'], label="Gender"),
    gr.inputs.Dropdown(choices=['Less than High School', 'High School', 'College'], label="Education"),
    gr.inputs.Checkbox(label="Current Smoker"),
    gr.inputs.Dropdown(choices=['Age Group 1', 'Age Group 2', 'Age Group 3'], label="Age Groups")
]

def predict_all(model_choice, *inputs):
    results = {}
    try:
        if 'Heart Disease' in model_choice:
            heart_input_data = inputs[:len(heart_inputs)]
            heart_pred = predict_heart(*heart_input_data)
            results['Heart Disease'] = heart_pred

        if 'Diabetes' in model_choice:
            diabetes_input_data = inputs[len(heart_inputs):len(heart_inputs) + len(diabetes_inputs)]
            diabetes_pred = predict_diabetes(*diabetes_input_data)
            results['Diabetes'] = diabetes_pred

        if 'Kidney Disease' in model_choice:
            kidney_input_data = inputs[len(heart_inputs) + len(diabetes_inputs):len(heart_inputs) + len(diabetes_inputs) + len(kidney_inputs)]
            kidney_pred = predict_kidney(*kidney_input_data)
            results['Kidney Disease'] = kidney_pred

        if 'Framingham Risk' in model_choice:
            framingham_input_data = inputs[len(heart_inputs) + len(diabetes_inputs) + len(kidney_inputs):]
            framingham_pred = predict_framingham(*framingham_input_data)
            results['Framingham Risk'] = framingham_pred

        return results
    except Exception as e:
        return {'error': str(e)}

# Combine all interfaces into one
app = gr.Interface(
    fn=predict_all,
    inputs=[model_choice] + heart_inputs + diabetes_inputs + kidney_inputs + framingham_inputs,
    outputs=gr.outputs.JSON(label="Predictions"),
    title="Health Prediction App",
    description="Select the models to use and provide inputs to get predictions."
)

app.launch()


C:\Users\Owner\anaconda3\lib\site-packages\gradio\inputs.py:149: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\Owner\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\Owner\anaconda3\lib\site-packages\gradio\inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\Owner\anaconda3\lib\site-packages\gradio\inputs.py:217: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\Owner\anaconda3\lib\site-packages\gradio\inputs.py:121: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, plea

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
